In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Heirarchy --- seasonality when sku level signal weak

In [13]:
df_raw = pd.read_excel("Hackathon Data.xlsx")

In [15]:
df_raw["Encoded_SKU_ID"].nunique()

575

In [16]:
df_raw.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7
3,4,2021-04-21,SO RANGEHOODS REG,KITCHEN VENTILATION,ML - Kitchen,VP - Major Appliances,549.99,?,498,Fully-Stocked,6
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7


In [18]:
df_raw["Inventory"].unique()

array(['Fully-Stocked', 'Constrained', 'Moderate', 'Out-of-Stock'],
      dtype=object)

In [19]:
df_raw['Encoded_SKU_ID'].isna().sum()

0

In [93]:
df_subset=df_raw[['Encoded_SKU_ID','SALES_DATE','Inventory','DAILY_UNITS']]

In [94]:
df_subset['Inventory'].isna().sum()

0

In [95]:
df_subset['DAILY_UNITS'].isna().sum()

0

In [96]:
df_subset['Inventory'].value_counts()

Fully-Stocked    692604
Moderate          54825
Constrained       52711
Out-of-Stock      46492
Name: Inventory, dtype: int64

In [97]:
df_subset["sales_yr"]=df_subset["SALES_DATE"].dt.year
df_subset["sales_quarter"]=df_subset["SALES_DATE"].dt.quarter

/var/folders/79/wlb93l7n51xbb60lcrj4jbr00000gn/T/ipykernel_67142/4032244470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["sales_yr"]=df_subset["SALES_DATE"].dt.year
/var/folders/79/wlb93l7n51xbb60lcrj4jbr00000gn/T/ipykernel_67142/4032244470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["sales_quarter"]=df_subset["SALES_DATE"].dt.quarter


In [98]:
def quarter(row):
    return 'Q'+ str(row.sales_quarter)+'_'+str(row.sales_yr)

In [99]:
df_subset.loc[:,'quarter']=df_subset.apply(lambda x:quarter(x), axis=1)

/var/folders/79/wlb93l7n51xbb60lcrj4jbr00000gn/T/ipykernel_67142/645980920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.loc[:,'quarter']=df_subset.apply(lambda x:quarter(x), axis=1)


In [35]:
df_subset.head()

,Encoded_SKU_ID,SALES_DATE,Inventory,DAILY_UNITS,sales_yr,sales_quarter,quarter
0,1,2017-09-28,Fully-Stocked,1,2017,3,Q3_2017
1,2,2017-11-24,Fully-Stocked,18,2017,4,Q4_2017
2,3,2019-11-28,Constrained,7,2019,4,Q4_2019
3,4,2021-04-21,Fully-Stocked,6,2021,2,Q2_2021
4,1,2018-10-27,Fully-Stocked,7,2018,4,Q4_2018


In [36]:
df_subset.columns

Index(['Encoded_SKU_ID', 'SALES_DATE', 'Inventory', 'DAILY_UNITS', 'sales_yr',
       'sales_quarter', 'quarter'],
      dtype='object')

In [100]:
df_g1=df_subset.groupby(["Encoded_SKU_ID","quarter"],as_index=False).\
agg(avg_quarterly_units_sold=('DAILY_UNITS','mean'),std_quarterly_units_sold=('DAILY_UNITS',lambda x: x.std(ddof=0)))

In [102]:
df_g1.columns

Index(['Encoded_SKU_ID', 'quarter', 'avg_quarterly_units_sold',
       'std_quarterly_units_sold'],
      dtype='object')

In [103]:
df_m1=df_subset.merge(df_g1,on=['Encoded_SKU_ID','quarter'],how="inner")

In [104]:
df_m1

,Encoded_SKU_ID,SALES_DATE,Inventory,DAILY_UNITS,sales_yr,sales_quarter,quarter,avg_quarterly_units_sold,std_quarterly_units_sold
0,1,2017-09-28,Fully-Stocked,1,2017,3,Q3_2017,2.695652,1.64673
1,1,2017-08-27,Fully-Stocked,2,2017,3,Q3_2017,2.695652,1.64673
2,1,2017-07-21,Fully-Stocked,6,2017,3,Q3_2017,2.695652,1.64673
3,1,2017-07-14,Out-of-Stock,6,2017,3,Q3_2017,2.695652,1.64673
4,1,2017-07-05,Out-of-Stock,2,2017,3,Q3_2017,2.695652,1.64673
...,...,...,...,...,...,...,...,...,...
846627,163,2019-09-29,Out-of-Stock,0,2019,3,Q3_2019,0.000000,0.00000
846628,563,2019-03-31,Out-of-Stock,0,2019,1,Q1_2019,0.000000,0.00000
846629,157,2017-12-28,Constrained,32,2017,4,Q4_2017,20.500000,11.50000
846630,157,2017-12-30,Constrained,9,2017,4,Q4_2017,20.500000,11.50000


In [105]:
def unusual_sales(row):
    if(row["avg_quarterly_units_sold"]-2*row["std_quarterly_units_sold"]<=row["DAILY_UNITS"]<=row["avg_quarterly_units_sold"]+2*row["std_quarterly_units_sold"]):
            return "Usual_sales"
    else:
        return "Unusual sales"

In [106]:
df_m1["sales_status"]=df_m1.apply(lambda x:unusual_sales(x),axis=1)

In [91]:
df_m1["Sales_status"].value_counts()

pandas.core.series.Series

In [123]:
df_g2=df_m1.groupby(["Encoded_SKU_ID","sales_status"],as_index=False).agg(Total_sales_sku=('DAILY_UNITS','sum'))

In [143]:
df_g2.groupby(["Encoded_SKU_ID"],as_index=False)["Total_sales_sku"].transform(sum)["Total_sales_sku"]

0        4273
1        4273
2       23783
3       23783
4        4468
        ...  
1145     6645
1146    19190
1147    19190
1148     6517
1149     6517
Name: Total_sales_sku, Length: 1150, dtype: int64

In [148]:
df_g2['%']=((df_g2["Total_sales_sku"])*100.0)/df_g2.groupby(["Encoded_SKU_ID"],as_index=False)["Total_sales_sku"]\
.transform(sum)["Total_sales_sku"]

In [165]:
th_1=df_g2[(df_g2["sales_status"]=="Unusual sales")]['%'].describe().loc['25%']
th_2=df_g2[(df_g2["sales_status"]=="Unusual sales")]['%'].describe().loc['75%']

In [186]:
n1=df_g2[(df_g2['%'].gt(20)) & (df_g2['sales_status']=="Unusual sales")].shape[0]
d1=df_g2['Encoded_SKU_ID'].nunique()

In [195]:
print(f"{n1} out of {d1} i.e. {(n1/d1)*100:.2f}% of SKU's have more than 20% of the daily units sold as unusual as per sales data ")

56 out of 575 i.e. 9.74% of SKU's have more than 20% of the daily units sold as unusual as per sales data 


In [193]:
df_g2[(df_g2['%'].gt(20)) & (df_g2['sales_status']=="Unusual sales")]["Encoded_SKU_ID"].unique()

array([ 15,  27,  37,  60,  70,  75,  98, 100, 114, 139, 150, 152, 155,
       160, 166, 169, 177, 242, 248, 254, 275, 278, 284, 292, 300, 314,
       316, 318, 329, 355, 360, 361, 362, 365, 383, 403, 405, 407, 409,
       428, 437, 443, 445, 455, 457, 471, 482, 483, 485, 503, 512, 517,
       542, 548, 551, 575])